In [1]:
# prepare for Latex Output
from IPython.display import HTML,Latex

# Colored output in Python

W  = '\033[0m'  # white (normal)
K  = '\033[30m' # black
R  = '\033[31m' # red
G  = '\033[32m' # green
O  = '\033[1;33m' # orange
B  = '\033[34m' # blue
P  = '\033[35m' # purple
T =  '\033[1;33;47m' #Title

# sympy functions and properties 

from sympy import symbols,pprint,integrate,diff,latex,simplify,Matrix,Abs,Ei,erf,erfc,E,Ne
from sympy import pi,sqrt,sin,cos,log,exp,oo,E,tan,Piecewise,asin,atan


from sympy.abc import rho,theta,phi
import matplotlib.pyplot as plt 
#
import numpy as np 
from mpl_toolkits.mplot3d import Axes3D


# visualisation by off-line plotly 

#import plotly.offline as pyo
#import plotly.express as ex
#import plotly.graph_objects as go

#pyo.init_notebook_mode()

In [14]:
# Variable declared
# x,y,z: variable Cartesian Corrdinates
# r,Theta: r,𝛉 in Polar Coordinates
# Rho,Theta,Phi: 𝛒, 𝛉, 𝛟 in Spherical Coordinates

x,y,z,r,t,rho,phi,u,v,a,b,c,m,n,l,p,T,𝝆,θ,𝝋=symbols("x y z r 𝜽 𝝆 𝝋 u v a b c m n l p T 𝝆 θ 𝝋")
r,Rho,Phi=symbols("r,rho,phi",positive=True)

a,b,c=symbols("a,b,c")

In [4]:
def Jacobian_det(XU,U):
    """
    Calculate the Jacobian, 𝛛X/𝛛U
    input: XU: original coordinates, X, in form of new coordinates, U,
           U: new Coordinates 
    output: absolute value of determinant of Jacobian       
    """
    
    MX=Matrix(XU)
    MU=Matrix(U)
    S= MX.jacobian(MU )     
    return simplify(Abs(S.det()))

In [1]:
# Fix error return of MultipleIntegral

def MultipleIntegral(f,X,XR):
    """
    input: ∫ dx ∫ dy ∫ f(x,y,z) dz
           f: f(x,y,z),
           X: (x,y, ...), variable pair
           XR: [[x0,x1], [y0,y1],...] 
    output: details of integral, enhanced by colored text, and value       
    """
    f_latex=latex(eval(str(f)))
    
    XR0=[]
    X_tex=[]
    for var in XR:
        XR0.append([latex(eval(str(var[0]))),latex(eval(str(var[1])))])
    for var in X:
        X_tex.append(latex(eval(str(var))))
    
    dA=""
    for var in X_tex:
        dA+=" d %s " %(var)
    if len(X)==2:
       intsign="\iint"
    elif(len(X)==3):
       intsign="\iiint" 
    else:
        intsign="\Large{\int}"    
    
    # indefinte I of z
    I='I'+str(X[-1])
    IX=[integrate(f,X[-1])]
    IX_tex=[latex(eval(str(IX[0])))]
    int_g='\left.\color{brown}{%s} \\right|_{%s}^{%s}' %(IX_tex[0],XR0[-1][0],XR0[-1][1])
    Integrand_tex_val=[int_g]

   
    Integrand=[f_latex]
    II='II'+str(X[-1])
    DIX=[integrate(f,(X[-1],XR[-1][0],XR[-1][1]))]
    DIX_tex=[latex(eval(str(DIX[0])))]
    n=len(X)
    i=0
    for var in X[len(X)-2::-1]:
        
        I='I'+str(var)
        while True:
            try:
                I=integrate(DIX[i],var)
                break
            except ValueError:
                #errortext="\\begin{eqnarray}"
                errortext="$$%s_{\Large{\mathbf{V}}} \color{brown}{%s} %s $$" %(intsign,f_latex,dA) 
                #errortext+="\end{eqnarray}"
                errortext+=("$\color{red}{\\text{Can't}}$ be integrated Here, Try another way ...")
                return Latex(errortext)
        #I=integrate(DIX[i],var)
        IX.append(I)
        IX_tex.append(latex(eval(str(I))))
        Integrand_tex_val.append('\left. \color{brown}{%s} \\right|_{%s}^{%s}' %(IX_tex[-1],XR0[n-i-2][0],XR0[n-i-2][1]) )
        II='II'+str(var)
        II=integrate(DIX[i],(var,XR[-i-2][0],XR[-i-2][1]))
        DIX.append(II)
        DIX_tex.append(latex(eval(str(II))))
        i+=1
        
    for I_f in DIX[:-1]:
        Integrand.append(latex(eval(str(I_f))))

    text0="\\begin{eqnarray}"
    text5="\end{eqnarray}"
    
    #domain="\large{\left\{"
    domain="\left\{\large{\\begin{array}{l}"
    i=0
    for var in X_tex:
        domain=domain+" %s \le %s \le %s,\cr" %(XR0[i][0],var,XR0[i][1])
        i+=1
    #domain+="\\right\}}"
    domain+="\end{array}}\\right\}"
    
        
    #dA=""
    #for var in X_tex:
    #    dA+=" d %s " %(var)
    
    textF="%s_{\Large{\mathbf{V}}=%s} \color{brown}{%s} dV&=& %s_{\Large{\mathbf{V}}} \color{brown}{%s} %s \cr " %(intsign,domain,f_latex,intsign,f_latex,dA)
    fubini="\color{blue}{\\text{Fubini's Theorem}}"
    
    textfI=""
    for i in range(len(X)):
        j=0
        if (i==0):
            eq1="%s&=&" %(fubini)
        else:
            eq1="&=&"
        textfI+=eq1  
        for var in X_tex[:len(X)-i-1:]:
            textfI+="\int_{%s}^{%s}d{%s}" %(XR0[j][0],XR0[j][1],var)
            j+=1
        #print(i,len(Integrand),len(X))  
        textfI+="\int_{%s}^{%s} \color{brown}{%s}d{%s}\cr "  %(XR0[-i-1][0],XR0[-i-1][1],Integrand[i],X_tex[-i-1])
        
        textfI+="&=&"
        int_pre=""

        for k in range(len(X)-i-2):
            int_pre+="\int_{%s}^{%s} d{%s}" %(XR0[k][0],XR0[k][1],X_tex[k])
        if len(X)-i-1>0: 
            int_ev="\int_{%s}^{%s} %s d{%s}" %(XR0[-i-2][0],XR0[-i-2][1],Integrand_tex_val[i],X_tex[-i-2])
        else:
            int_ev="%s" %(Integrand_tex_val[-1])
        textfI+=int_pre+int_ev+'\cr'
    textfI+='&=&%s' %(DIX_tex[-1])   

    text=text0+textF+textfI+text5

    return Latex(text)

1. $\int_0^2 \int_0^{\sqrt{2x-x^2}} \sqrt{x^2 + y^2} dy \, dx.$
2. $\int_0^1 \int_0^{\sqrt{1-x^2}} (x^2 + y^2) dy \, dx,$
3. $\iint_R (x - y) dy \, dx,$ where $R$ is of the parallelogram are region bound by $x-y=-1,x-y=1,x-3y=-5, x-3y=-9$
4. $\iint_R \frac{4}{(x - y)^2} dA,$, where $R$ is trapezoidal bounded by the lines, $x=0,y=0,x-y=2,x-y=4$.
5. $\iint_R (x + y)e^{x^2-y^2} dA$ where $R$ is region bounded by $x+y=1, x+y=3, x^2+y^2=\pm1$
6. $\displaystyle\iint_R y \, \sin (y^2 - x) \,dA,$ where $R$ is region bounded by $y=0,x=2,y=\sqrt x$.
7. $\int_0^3 \int_0^4 \int_{y/2}^{(y/2)+1} \left(x + \frac{z}{3}\right) dx \, dy \, dz$
8. Area bounded by $x^4+y^4=1$
9. $\iiint_V (x^2 y + 3xyz) \, dx \, dy \, dz$ where $V$ is the solid bounded by $1\le x\le2,0\le z\le1, 0\le xy\le1$.
10. $\int_0^{2\pi} \int_0^1 \int_0^1 dz \, dr \, d\theta$
11. $\iint_R (5x^3y^2 - y^2) \, dA, \, R = \{(x,y)|0 \leq x \leq 2, \, 1 \leq y \leq 4\}$
12. $\iint_D \frac{y}{3x^2 + 1} dA, \, D = \{(x,y) |0 \leq x \leq 1, \, -x \leq y \leq x\}$
13. $\iint_D \sin (x^2 + y^2) dA$ where $D$ is unit circle.
14. $\int_0^1 \int_0^1 xye^{x^2} dx \, dy=1/4$
15. $\int_{-1}^1 \int_0^z \int_0^{x-z} 6dy \, dx \, dz$
16. $\iiint_V 3y \, dV,$ where $V = \{(x,y,z) |0 \leq x \leq 1, \, 0 \leq y \leq x, \, 0 \leq z \leq \sqrt{9 - y^2}\}=1.475$
17. $\int_0^2 \int_0^{2\pi} \int_r^1 r \, dz \, d\theta \, dr$
18. $\int_0^{2\pi} \int_0^{\pi/2} \int_1^3 \rho^2 \, \sin(\varphi) d\rho \, d\varphi, \, d\theta=52\pi/3$
19. $\int_0^1 \int_{-\sqrt{1-x^2}}^{\sqrt{1-x^2}} \int_{-\sqrt{1-x^2-y^2}}^{\sqrt{1-x^2-y^2}} dz \, dy \, sx$
20. The volume of the solid that lies between the paraboloid $z=2x^2+2y^2$ and $z=8$. 
21. Volume of solid bounded by $x^2+y^2=16$, $z=1$ and $z+x=2$ 93.291.
22. The volume of the intersection between two spheres of radius 1, the top whose center is  (0,0,0.25)
 and the bottom, which is centered at  (0,0,0).
23. solid bounded by $z=\sqrt(x^2+y^2)$ and $x^2+y^2+z^2=z$.
24. 



In [11]:
#1

f=r
J=r
X=[t,r]
XR=[[0,pi/2],[0,2*cos(t)]]

MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [13]:
#2 

f=r*r
J=r
X=[t,r]
XR=[[0,pi/2],[0,1]]

MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [15]:
#3 U=x-y, V=x-3y, x=(3U-V/2), y=(U-V)/2

f=u
J=1/2
X=[u,v]
XR=[[-1,1],[-9,-5]]

MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [17]:
#4 U=x-y, V=x+y, x=(U+V/2), y=(U-V)/2

f=4/u**2
J=1/2
X=[v,u]
XR=[[2,4],[-v,v]]

MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [20]:
#5 U=x-y, V=x+y, x=(U+V)/2,y=(V-U)/2

f=v*exp(u*v)
J=1/2
X=[v,u]

XR=[[1,3],[-1/v,1/v]]
MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [28]:
#6 U=x^2-x, V=x, x=V,y=sqrt(u+v)

f=sqrt(u+v)*sin(u)
J=1/2/sqrt(u+v)
X=[v,u]

XR=[[0,2],[-v,0]]
MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [33]:
#7 U=x^4, V=y^4, x=U^(1/4),y=V**(1/4)

f=1
J=(u*v)**(-3/4)/16
X=[v,u]

XR=[[0,1],[0,1]]
MultipleIntegral(f*J,X,XR)

<IPython.core.display.Latex object>

In [30]:
# 8 
f=(x+z/3)
X=[z,y,x]
XR=[[0,3],[0,4],[y/2,1+y/2]]
MultipleIntegral(f,X,XR)

<IPython.core.display.Latex object>

In [30]:
# 7 
f=(x+z/3)
X=[z,y,x]
XR=[[0,3],[0,4],[y/2,1+y/2]]
MultipleIntegral(f,X,XR)

<IPython.core.display.Latex object>

In [30]:
# 7 
f=(x+z/3)
X=[z,y,x]
XR=[[0,3],[0,4],[y/2,1+y/2]]
MultipleIntegral(f,X,XR)

<IPython.core.display.Latex object>

In [32]:
# 9
f=x**2*y+3*x*y*z
X=[z,x,y]
XR=[[0,1],[1,2],[0,2/x]]
MultipleIntegral(f,X,XR)

<IPython.core.display.Latex object>

In [7]:
f=r
X=[r,Theta,z]
XR=[[0,sqrt(2)],[0,2*pi],[0,r**2,sqrt(6-r**2)]]

MultipleIntegral(f,X,XR)

<IPython.core.display.Latex object>

In [5]:
f=Rho**2*sin(Phi)*2*2*4
X=[Theta,Phi,Rho]
XR=[[0,2*pi],[0,pi],[0,1]]

MultipleIntegral(f,X,XR)

<IPython.core.display.Latex object>